In [1]:
# 데이터는 CIFAR10 10가지 분류....
# 모델을만들어서
# 분류하고자한는 이미지가 폴더에 있을때. 해당 폴더로 자동으로 분류(이동)

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50

In [3]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
# 정규화
x_train = x_train / 255.0
x_test = x_test / 255.0

170498071/170498071 [==============================] - 2s 0us/step


In [4]:
x_train.shape[1:]

(32, 32, 3)

In [5]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [ ]:
from tensorflow.python.framework import tensor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape = x_train.shape[1:])
# 기존 모델의 가중치를 동결
for layer in base_model.layers:
  layer.trainable = False

# 마지막에 우리 분류기 넣는다.
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256,activation='relu'),
    layers.Dense(10,activation='softmax')
])
model.compile(optimizer='adam', loss = tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
1233/1563 [======================>.......] - ETA: 3s - loss: 2.0814 - accuracy: 0.2377